# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

In [1]:
! pip install -q "datasets==2.15.0"

In [2]:
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification

from peft import LoraConfig
from peft import get_peft_model

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [3]:
# get dataset for sms spam
dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=2)
splits = ["train", "test"]

# create tokenizer instance
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token 

# tokenize all dataset
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], truncation=True), batched=True
    )


Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

In [4]:
# create model instance
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

# freeze all the model parameters.
for param in model.parameters():
    param.requires_grad = False

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Set parameter for model
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam",
        learning_rate=5e-6,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,        
        evaluation_strategy="epoch",
        save_strategy="epoch",        
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [7]:
foundation_eval_results= trainer.evaluate()
foundation_eval_results

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 7.175482273101807,
 'eval_accuracy': 0.12286995515695068,
 'eval_runtime': 18.5931,
 'eval_samples_per_second': 59.968,
 'eval_steps_per_second': 59.968}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [8]:
from peft import get_peft_model
from peft import LoraConfig, TaskType

config =  LoraConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, r=1, lora_alpha=1, lora_dropout=0.1)
model_name_or_path = model_name

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1}, 
    return_dict=True)

lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 39,936 || all params: 124,479,744 || trainable%: 0.03208232818987802


In [9]:
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam_PEFT",
        learning_rate=5e-6,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,        
        evaluation_strategy="epoch",
        save_strategy="epoch",        
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.544800,1.338538,0.879821


Checkpoint destination directory ./data/spam_not_spam_PEFT/checkpoint-4459 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=4459, training_loss=1.1971455599914673, metrics={'train_runtime': 225.9743, 'train_samples_per_second': 19.732, 'train_steps_per_second': 19.732, 'total_flos': 54413575123968.0, 'train_loss': 1.1971455599914673, 'epoch': 1.0})

In [11]:
lora_model.save_pretrained("gpt-lora")

In [12]:
lora_eval_results = trainer.evaluate()

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [19]:
lora_loded_model = AutoModelForSequenceClassification.from_pretrained(
    "gpt-lora",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)


trainer = Trainer(
    model=lora_loded_model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam_PEFT",
        learning_rate=5e-6,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,        
        evaluation_strategy="epoch",
        save_strategy="epoch",        
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

lora_loded_eval_results=trainer.evaluate()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
print("Compare Pre-trained and PEFT(LORA) Models:")
print("Pre-trained Model:", foundation_eval_results['eval_accuracy'])
print("PEFT Model:", lora_loded_eval_results['eval_accuracy'])

Compare Pre-trained and PEFT(LORA) Models:
Pre-trained Model: 0.12286995515695068
PEFT Model: 0.8798206278026905
